In [1]:
import librosa
import librosa.display
import os
from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import random
import keras
import datetime as dt
from datetime import datetime
from keras import layers
from keras import models
from keras import optimizers
from keras.applications.densenet import DenseNet121, DenseNet169
#from keras.applications.nasnet import NASNetLarge
# from keras_efficientnets import EfficientNetB7
from keras.layers import *
from keras.models import *
from keras import initializers, regularizers, constraints
from keras.layers.normalization import BatchNormalization 
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from keras import backend as K

tf.set_random_seed(1234)
random.seed(100)

os.environ["CUDA_VISIBLE_DEVICES"]="1"
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

def score_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

def dot_product(x, kernel):
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)
    
class AttentionWithContext(Layer):
    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')
        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)
        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)
        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3
        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
            self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)
        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)
        if self.bias:
            uit += self.b
        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)
        a = K.exp(ait)
        if mask is not None:
            a *= K.cast(mask, K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

    
def cce_f1_loss(y_true, y_pred):
    return 1 + 0.1*keras.losses.categorical_crossentropy(y_true, y_pred) - keras.metrics.categorical_accuracy(y_true, y_pred)

# Find unique number of classes  
def get_unique_classes(input_directory,files):

    unique_classes=set()
    for f in files:
        g = f.replace('.mat','.hea')
        input_file = os.path.join(input_directory,g)
        with open(input_file,'r') as f:
            for lines in f:
                if lines.startswith('#Dx'):
                    tmp = lines.split(': ')[1].split(',')
                    for c in tmp:
                        unique_classes.add(c.strip())
    return sorted(unique_classes)

def one_hot_encoding(one_hot_vector,y, class2index):
    ind=class2index[y]
    one_hot_vector[ind]=1
    return one_hot_vector

# Search for multi-label subjects
def searching_overlap(input_directory,class2index, input_file_names):
    multiclasses=[]
    multisubjects=[]
    number = []
    for file in input_file_names:
        f=file
        g = f.replace('.mat','.hea')
        input_file = os.path.join(input_directory,g)
        with open(input_file,'r') as f:
            for lines in f:
                if lines.startswith('#Dx'):
                    tmp = lines.split(': ')[1].split(',')
                    if len(tmp)>1:
                        one_hot_vector = [0]*(len(class2index))
                        for c in tmp:
                            one_hot_vector = one_hot_encoding(one_hot_vector, c.strip(), class2index)
                        multiclasses.append(one_hot_vector)
                        multisubjects.append(g)
                        number.append(len(tmp))
    return multisubjects, multiclasses, number

def block_feature(sequence_en, minimum_len): 
    new_en = []
    if len(sequence_en) > minimum_len:  # 길이가 minimum보다 긴 경우
        start = random.randint(0,len(sequence_en)-minimum_len)
        #print(start)
        new_en = sequence_en[start:start+minimum_len]
    elif len(sequence_en) == minimum_len: # 길이가 minimum
        new_en = sequence_en
    else: 
        assert len(sequence_en) <= minimum_len
    return new_en

def exploratory_look(input_directory,file, class2index):
    classes = []
    f = file
    g = f.replace('.mat','.hea')
    input_file = os.path.join(input_directory,g)
    with open(input_file,'r') as f:
        for lines in f:
            if lines.startswith('#Dx'):
                tmp = lines.split(': ')[1].split(',')
                print(tmp, len(tmp))
    return tmp     

# Get classes of sorted file names
def get_labels(input_directory,file, class2index):
    f = file
    g = f.replace('.mat','.hea')
    input_file = os.path.join(input_directory,g)
    with open(input_file,'r') as f:
        for lines in f:
            if lines.startswith('#Dx'):
                tmp = lines.split(': ')[1].split(',')
                one_hot_vector = [0]*(len(class2index))
                for c in tmp:
                    one_hot_vector = one_hot_encoding(one_hot_vector, c.strip(), class2index)
                
    return one_hot_vector

def randextract_mels(curr_step, batch_size, data, mel_directory, class2index, minimum_len, x_mean_final, x_std_final):
    mel_files = []
    classes = []
    start = batch_size*curr_step
    end = batch_size*(curr_step+1)
    curr_file_indices = data[start:end]
    for file in curr_file_indices:
        tmp_file = np.load(mel_directory + '/' + file.replace('.mat', '.npy'))
        clip_file = block_feature(tmp_file, minimum_len)
        #print(clip_file.shape)
        #clip_file = tmp_file[:minimum_len]
        clip_file -= x_mean_final
        clip_file /= x_std_final
        mel_files.append(clip_file)
        label = get_labels(input_directory, file, class2index)
        classes.append(label)
    concat = list(zip(mel_files, classes))
    random.shuffle(concat)
    mel_files, classes = zip(*concat)
    return mel_files, classes

def train(data_train, mel_directory, batch_size, class2index, minimum_len, x_mean_final, x_std_final): 
    loss=[]
    acc = []
    f1 = []

    total_steps = int(np.ceil(len(data_train)/batch_size))
    for curr_step in range(total_steps):
        batch_mels, batch_labels = randextract_mels(curr_step, batch_size, data_train, mel_directory, class2index, minimum_len, x_mean_final, x_std_final)
        batch_mels = np.asarray(batch_mels)
        batch_labels = np.asarray(np.squeeze(batch_labels))
        train_tmp = model.train_on_batch(batch_mels, batch_labels)
        loss.append(train_tmp[0])
        acc.append(train_tmp[1])
        f1.append(train_tmp[2])

    loss = np.mean(np.array(loss))
    acc = np.mean(np.array(acc))
    f1 = np.mean(np.array(f1))
    return loss, acc, f1

def test(data, mel_directory, input_directory, class2index, minimum_len, model, x_mean_final, x_std_final):
    scores = []
    predicted_labels=[]
    accuracy=np.zeros(len(data))
    #total_loss=[]
    total_acc = 0
    total_f1 = 0
    
    mel_files = []
    classes = []
    for i, file in enumerate(data):
        tmp_file = np.load(mel_directory + '/' + file.replace('.mat', '.npy'))
        steps = int(np.floor(tmp_file.shape[0]/minimum_len))
        mel_files = []
        for block in range(steps): # 128개씩 쪼갠 블럭 단위로 predict
            start = block*minimum_len
            end = (block+1)*minimum_len
            clip_file = tmp_file[start:end]
            clip_file -= x_mean_final
            clip_file /= x_std_final
            mel_files.append(clip_file)
        mel_files = np.asarray(mel_files)
        logit = model.predict(mel_files)
        logit = np.mean(logit, axis=0)
        
        
        
        pred = np.zeros(len(logit))
        for ii, label in enumerate(logit):
            if label >= 0.5:
                pred[ii] = 1
            else:
                pred[ii] = 0
        pred = pred.tolist()
        label = get_labels(input_directory,file,class2index)

        
        if pred == label:
            acc = 1
        else:
            acc = 0
        total_acc += acc
        #total_f1 += f1
    final_acc = total_acc / i
    #final_f1 = total_f1 / i
    return final_acc#, final_f1

batch_size = 32
minimum_len = 2880
epochs = 1000
loss_function = 'categorical_crossentropy'
activation_function = 'softmax'
rootdir = '../'
date = datetime.today().strftime("%Y%m%d")
input_directory = os.path.join(rootdir, 'Training_WFDB')
mel_name = 'Raw_data_20200424' 
mel_directory = os.path.join(rootdir, mel_name)
results_directory = os.path.join(rootdir, 'results_'+date+'_0')
if not os.path.isdir(input_directory):
    os.mkdir(input_directory)
if not os.path.isdir(mel_directory):
    os.mkdir(mel_directory)
# if not os.path.isdir(results_directory):
#     os.mkdir(results_directory)
        
input_files = []
for f in os.listdir(input_directory):
    if os.path.isfile(os.path.join(input_directory, f)) and not f.lower().startswith('.') and f.lower().endswith('mat'):
        input_files.append(f)
input_file_names = sorted(input_files)

unique_classes = get_unique_classes(input_directory, input_files)
class2index = {}
for a, b in enumerate(unique_classes):
    class2index[b] = a
    
# classes_orig= [x.replace('.mat', '.hea') for x in input_file_names] # total subjects
# classes_multi, _, _ = searching_overlap(input_directory,class2index, input_file_names)
# classes_single = [x for x in classes_orig if x not in classes_multi]
# classes_single = [x.replace('.hea', '.mat') for x in classes_single]

# double-checking if classes_single have single-label
# a, b, c  = searching_overlap(input_directory,class2index,classes_single)

# we can safely use classes_single as input_file_names
# input_file_names = classes_single
random.shuffle(input_file_names)
np.shape(input_file_names)

x_mean_all = []
x_std_all = []
for file in input_file_names:
    x = np.load(mel_directory + '/' + file.replace('.mat', '.npy'))
    x_mean = [np.mean(x[:,0]), np.mean(x[:,1]), np.mean(x[:,2]), np.mean(x[:,3]), np.mean(x[:,4]), np.mean(x[:,5]),
             np.mean(x[:,6]), np.mean(x[:,7]), np.mean(x[:,8]), np.mean(x[:,9]), np.mean(x[:,10]), np.mean(x[:,11])]
    
    x_std = [np.std(x[:,0]), np.std(x[:,1]), np.std(x[:,2]), np.std(x[:,3]), np.std(x[:,4]), np.std(x[:,5]),
             np.std(x[:,6]), np.std(x[:,7]), np.std(x[:,8]), np.std(x[:,9]), np.std(x[:,10]), np.std(x[:,11])]
    #print(x_mean)
    x_mean_all.append(x_mean)
    x_std_all.append(x_std)
x_mean_final = np.mean(x_mean_all, axis=0)
x_std_final = np.mean(x_std_all, axis=0)
print(x_mean_final)

data, data_test = train_test_split(input_file_names, test_size = 0.2, train_size = 0.8, shuffle=True)
# data_train, data_val = train_test_split(data, test_size = 0.25, train_size = 0.75, shuffle=True)
data_train, data_val = train_test_split(data, test_size = 0.05, train_size = 0.95, shuffle=True)
print(np.shape(data_train), np.shape(data_val), np.shape(data_test))

main_input = Input(shape=(minimum_len,12), dtype='float32', name='main_input')

branch_pred = []
for i in range(12):
    # Slicing the ith channel:
    input_sl = Lambda(lambda x: x[:, :, i:i+1])(main_input)
    #print(input_sl)
    x1 = GaussianNoise(0.01 ,input_shape=(minimum_len, 1))(input_sl)
    x1 = Conv1D(32, 10, dilation_rate=3, padding='same')(x1)
    x1 = BatchNormalization()(x1)
    x1 = LeakyReLU(alpha=0.3)(x1)
    x2 = Conv1D(32, 10, dilation_rate=3, padding='same')(x1)
    x2 = add([x2, x1])
    x2 = BatchNormalization()(x2)
    x2 = LeakyReLU(alpha=0.3)(x2)
    #x2 = MaxPooling1D(pool_size=2)(x2)
    x2 = Convolution1D(32, 20, strides = 2, padding='same')(x2)
    #x2 = Dropout(0.25)(x2)

    x3 = Conv1D(64, 10, dilation_rate=2, padding='same')(x2)
    x3 = BatchNormalization()(x3)
    x3 = LeakyReLU(alpha=0.3)(x3)
    x4 = Conv1D(64, 10, dilation_rate=2, padding='same')(x3)
    x4 = add([x4, x3])
    x4 = BatchNormalization()(x4)
    x4 = LeakyReLU(alpha=0.3)(x4)
    #x4 = MaxPooling1D(pool_size=2)(x4)
    x4 = Convolution1D(32, 20, strides = 2, padding='same')(x4)
    #x4 = Dropout(0.25)(x4)

    x5 = Conv1D(128, 10, dilation_rate=1, padding='same')(x4)
    x5 = BatchNormalization()(x5)
    x5 = LeakyReLU(alpha=0.3)(x5)
    x6 = Conv1D(128, 10, dilation_rate=1, padding='same')(x5)
    x6 = add([x6, x5])
    x6 = BatchNormalization()(x6)
    x6 = LeakyReLU(alpha=0.3)(x6)
    #x6 = MaxPooling1D(pool_size=2)(x6)
    x6 = Convolution1D(64, 20, strides = 2, padding='same')(x6)
    #x6 = Dropout(0.25)(x6)
    '''
    x7 = Conv1D(64, 10, dilation_rate=1, padding='same')(x6)
    x7 = BatchNormalization()(x7)
    x7 = LeakyReLU(alpha=0.3)(x7)
    x8 = Conv1D(64, 10, dilation_rate=1, padding='same')(x7)
    x8 = add([x8, x7])
    x8 = BatchNormalization()(x8)
    x8 = LeakyReLU(alpha=0.3)(x8)
    #x8 = MaxPooling1D(pool_size=2)(x8)
    x8 = Convolution1D(64, 20, strides = 2, padding='same')(x8)
    #cnnout = GlobalAveragePooling1D()(x8)
    #x8 = Dropout(0.25)(x8)
    '''
    x = Bidirectional(CuDNNGRU(128, input_shape=(360,128),return_sequences=True,return_state=False))(x6)
    x = LeakyReLU(alpha=0.3)(x)
    x = AttentionWithContext()(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.3)(x)
    
    #x = GlobalAveragePooling1D()(x8)
    #x = Flatten()(x8)
    #x = Dense(512)(x)
    #x = BatchNormalization()(x)
    #x = Activation('relu')(x)
    #x = Dropout(0.5)(x)
    x = Dense(9)(x)
#     pred  = Activation('softmax')(x)
    pred  = Activation('sigmoid')(x)
    branch_pred.append(pred)
    
out = Average()(branch_pred)
#print(out)

model = Model(inputs=main_input, outputs=out)
model.summary()

model.compile(loss=loss_function,
              optimizer=optimizers.Adam(lr=1e-5),
              metrics=['acc', score_f1])

val_acc_sum=[]
train_loss_sum=[]
train_acc_sum=[]
val_loss_sum=[]
val_acc_min = 0
for num_epoch in range(epochs):
    random.shuffle(data_train)
    train_loss, train_acc, train_f1 = train(data_train, mel_directory, batch_size, class2index, minimum_len, x_mean_final, x_std_final)
    print('\nEpoch',num_epoch+1,'train_loss:',f'{train_loss:.3f}','train_acc:',f'{train_acc:.3f}','train_f1:',f'{train_f1:.3f}',"\t")
    model_output = "ecg_mel_E%02dL%.2f" % (num_epoch, train_loss)
    save_name = os.path.join(results_directory, model_output)
    val_acc = test(data_val, mel_directory, input_directory, class2index, minimum_len, model, x_mean_final, x_std_final)
    if val_acc > val_acc_min:
        val_acc_min = val_acc
#         model.save(save_name)
    print('\nValidation', num_epoch+1, 'valid_acc:',f'{val_acc:.3f}', 'best_acc:',f'{val_acc_min:.3f}', "\t")


/usr/local/lib/python3.6/dist-packages/numba/errors.py:137: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)typ

[-0.0012702   0.00038759  0.00402317 -0.0029567   0.00051957  0.00116427
 -0.0008405  -0.00222516 -0.0019119  -0.00093502  0.00129789  0.00177598]
(5225,) (276,) (1376,)





__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, 2880, 12)     0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 2880, 1)      0           main_input[0][0]                 
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 2880, 1)      0           main_input[0][0]                 
__________________________________________________________________________________________________
lambda_3 (Lambda)               (

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1 train_loss: 2.133 train_acc: 0.481 train_f1: 0.341 	

Validation 1 valid_acc: 0.116 best_acc: 0.116 	

Epoch 2 train_loss: 1.981 train_acc: 0.566 train_f1: 0.388 	

Validation 2 valid_acc: 0.138 best_acc: 0.138 	

Epoch 3 train_loss: 1.903 train_acc: 0.602 train_f1: 0.410 	

Validation 3 valid_acc: 0.153 best_acc: 0.153 	

Epoch 4 train_loss: 1.852 train_acc: 0.618 train_f1: 0.424 	

Validation 4 valid_acc: 0.160 best_acc: 0.160 	

Epoch 5 train_loss: 1.805 train_acc: 0.631 train_f1: 0.439 	

Validation 5 valid_acc: 0.171 best_acc: 0.171 	

Epoch 6 train_loss: 1.766 train_acc: 0.646 train_f1: 0.450 	

Validation 6 valid_acc: 0.185 best_acc: 0.185 	

Epoch 7 train_loss: 1.730 train_acc: 0.653 train_f1: 0.467 	

Validation 7 valid_acc: 0.182 best_acc: 0.185 	

Epoch 8 train_loss: 1.698 train_acc: 0.656 train_f1: 0.481 	

Validation 8 valid_acc: 0.189 best_acc: 0.189 	

Epoch 9 train_los


Epoch 71 train_loss: 0.822 train_acc: 0.776 train_f1: 0.666 	

Validation 71 valid_acc: 0.447 best_acc: 0.629 	

Epoch 72 train_loss: 0.811 train_acc: 0.775 train_f1: 0.672 	

Validation 72 valid_acc: 0.480 best_acc: 0.629 	

Epoch 73 train_loss: 0.813 train_acc: 0.776 train_f1: 0.658 	

Validation 73 valid_acc: 0.425 best_acc: 0.629 	

Epoch 74 train_loss: 0.792 train_acc: 0.785 train_f1: 0.658 	

Validation 74 valid_acc: 0.411 best_acc: 0.629 	

Epoch 75 train_loss: 0.793 train_acc: 0.782 train_f1: 0.648 	

Validation 75 valid_acc: 0.425 best_acc: 0.629 	

Epoch 76 train_loss: 0.785 train_acc: 0.780 train_f1: 0.640 	

Validation 76 valid_acc: 0.396 best_acc: 0.629 	

Epoch 77 train_loss: 0.775 train_acc: 0.784 train_f1: 0.629 	

Validation 77 valid_acc: 0.378 best_acc: 0.629 	

Epoch 78 train_loss: 0.766 train_acc: 0.786 train_f1: 0.621 	

Validation 78 valid_acc: 0.393 best_acc: 0.629 	

Epoch 79 train_loss: 0.768 train_acc: 0.786 train_f1: 0.609 	

Validation 79 valid_acc: 0.415 b


Epoch 143 train_loss: 0.457 train_acc: 0.878 train_f1: 0.277 	

Validation 143 valid_acc: 0.142 best_acc: 0.629 	

Epoch 144 train_loss: 0.467 train_acc: 0.876 train_f1: 0.267 	

Validation 144 valid_acc: 0.120 best_acc: 0.629 	

Epoch 145 train_loss: 0.456 train_acc: 0.873 train_f1: 0.263 	

Validation 145 valid_acc: 0.124 best_acc: 0.629 	

Epoch 146 train_loss: 0.456 train_acc: 0.875 train_f1: 0.268 	

Validation 146 valid_acc: 0.127 best_acc: 0.629 	

Epoch 147 train_loss: 0.460 train_acc: 0.874 train_f1: 0.269 	

Validation 147 valid_acc: 0.120 best_acc: 0.629 	

Epoch 148 train_loss: 0.448 train_acc: 0.880 train_f1: 0.262 	

Validation 148 valid_acc: 0.124 best_acc: 0.629 	

Epoch 149 train_loss: 0.448 train_acc: 0.875 train_f1: 0.265 	

Validation 149 valid_acc: 0.124 best_acc: 0.629 	

Epoch 150 train_loss: 0.441 train_acc: 0.877 train_f1: 0.260 	

Validation 150 valid_acc: 0.124 best_acc: 0.629 	

Epoch 151 train_loss: 0.441 train_acc: 0.884 train_f1: 0.256 	

Validation 151 


Validation 214 valid_acc: 0.062 best_acc: 0.629 	

Epoch 215 train_loss: 0.287 train_acc: 0.928 train_f1: 0.150 	

Validation 215 valid_acc: 0.062 best_acc: 0.629 	

Epoch 216 train_loss: 0.305 train_acc: 0.916 train_f1: 0.138 	

Validation 216 valid_acc: 0.058 best_acc: 0.629 	

Epoch 217 train_loss: 0.295 train_acc: 0.921 train_f1: 0.149 	

Validation 217 valid_acc: 0.069 best_acc: 0.629 	

Epoch 218 train_loss: 0.289 train_acc: 0.923 train_f1: 0.150 	

Validation 218 valid_acc: 0.062 best_acc: 0.629 	

Epoch 219 train_loss: 0.284 train_acc: 0.926 train_f1: 0.149 	

Validation 219 valid_acc: 0.069 best_acc: 0.629 	

Epoch 220 train_loss: 0.290 train_acc: 0.922 train_f1: 0.142 	

Validation 220 valid_acc: 0.069 best_acc: 0.629 	

Epoch 221 train_loss: 0.282 train_acc: 0.929 train_f1: 0.141 	

Validation 221 valid_acc: 0.055 best_acc: 0.629 	

Epoch 222 train_loss: 0.278 train_acc: 0.930 train_f1: 0.130 	

Validation 222 valid_acc: 0.062 best_acc: 0.629 	

Epoch 223 train_loss: 0.272 


Epoch 286 train_loss: 0.208 train_acc: 0.943 train_f1: 0.050 	

Validation 286 valid_acc: 0.011 best_acc: 0.629 	

Epoch 287 train_loss: 0.220 train_acc: 0.940 train_f1: 0.046 	

Validation 287 valid_acc: 0.007 best_acc: 0.629 	

Epoch 288 train_loss: 0.207 train_acc: 0.947 train_f1: 0.038 	

Validation 288 valid_acc: 0.011 best_acc: 0.629 	

Epoch 289 train_loss: 0.204 train_acc: 0.947 train_f1: 0.045 	

Validation 289 valid_acc: 0.015 best_acc: 0.629 	

Epoch 290 train_loss: 0.213 train_acc: 0.945 train_f1: 0.047 	

Validation 290 valid_acc: 0.007 best_acc: 0.629 	

Epoch 291 train_loss: 0.214 train_acc: 0.940 train_f1: 0.043 	

Validation 291 valid_acc: 0.018 best_acc: 0.629 	

Epoch 292 train_loss: 0.220 train_acc: 0.941 train_f1: 0.048 	

Validation 292 valid_acc: 0.015 best_acc: 0.629 	

Epoch 293 train_loss: 0.216 train_acc: 0.940 train_f1: 0.042 	

Validation 293 valid_acc: 0.018 best_acc: 0.629 	

Epoch 294 train_loss: 0.204 train_acc: 0.943 train_f1: 0.039 	

Validation 294 


Validation 357 valid_acc: 0.000 best_acc: 0.629 	

Epoch 358 train_loss: 0.170 train_acc: 0.956 train_f1: 0.004 	

Validation 358 valid_acc: 0.004 best_acc: 0.629 	

Epoch 359 train_loss: 0.168 train_acc: 0.952 train_f1: 0.004 	

Validation 359 valid_acc: 0.004 best_acc: 0.629 	

Epoch 360 train_loss: 0.172 train_acc: 0.954 train_f1: 0.004 	

Validation 360 valid_acc: 0.004 best_acc: 0.629 	

Epoch 361 train_loss: 0.167 train_acc: 0.952 train_f1: 0.002 	

Validation 361 valid_acc: 0.004 best_acc: 0.629 	

Epoch 362 train_loss: 0.164 train_acc: 0.955 train_f1: 0.002 	

Validation 362 valid_acc: 0.000 best_acc: 0.629 	

Epoch 363 train_loss: 0.172 train_acc: 0.952 train_f1: 0.003 	

Validation 363 valid_acc: 0.000 best_acc: 0.629 	

Epoch 364 train_loss: 0.172 train_acc: 0.949 train_f1: 0.001 	

Validation 364 valid_acc: 0.007 best_acc: 0.629 	

Epoch 365 train_loss: 0.170 train_acc: 0.956 train_f1: 0.002 	

Validation 365 valid_acc: 0.004 best_acc: 0.629 	

Epoch 366 train_loss: 0.171 

KeyboardInterrupt: 

In [ ]:
print('done')